In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

#import the web scraper
import urllib.request
!pip install beautifulsoup4
!pip3 install lxml
from bs4 import BeautifulSoup

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

/bin/sh: 1: pip3: not found
Libraries imported.


In [2]:
!pip install lxml

In [3]:
#scrape the web and foramt the data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page)
soup.prettify()
##maeke the df
table = soup.find_all('table')
df = pd.read_html(str(table))[0]

In [4]:
!pip3 install lxml

/bin/sh: 1: pip3: not found


In [5]:
#get rid of the NA data in Borough
dropNAS=df[df['Borough']=='Not assigned'].index
df.drop(dropNAS, inplace=True)
#check our data. are there any NAs?
print(df.isna().sum())#check if there are any NAs in Neighborhoods
df=df.reset_index()

print(df.shape)

Postal Code     0
Borough         0
Neighborhood    0
dtype: int64
(103, 4)


In [6]:
df=df.drop(['index'], axis=1)
print(df.head())

  Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


In [7]:
#get the csv files
df_loc=pd.read_csv('http://cocl.us/Geospatial_data')
df_loc.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [8]:
#join the data frames on Postal Code
df=pd.merge(df, df_loc, on='Postal Code')
df.head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [9]:
#check to see if my data corralates to the exmaples
print(df.loc[df['Postal Code']=='M5G'])
print(df.dtypes)

   Postal Code           Borough        Neighborhood   Latitude  Longitude
24         M5G  Downtown Toronto  Central Bay Street  43.657952 -79.387383
Postal Code      object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object


In [10]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent='Canada_Explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronoto are 43.6534817, -79.3839347.


In [11]:
#making a list and checking it twice
locations = df[['Latitude', 'Longitude']]
locations_as_list = locations.values.tolist()
len(locations_as_list)

103

In [12]:
to_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for i in range(0, len(locations_as_list)):
    folium.CircleMarker(locations_as_list[i], popup=df['Postal Code'][i], radius = 5, color = 'green', fill = True, fill_color = 'green').add_to(to_map)

In [13]:
to_map